# generateLigandDistanceAndSimilarities


Carl's SOAPXX code outputs a .hdf5 file (from a .xyz file input) giving the labels of the processed files. I've modified it to output in a "basematrices" directory the K_base and P_base for each protein. From that, we can work out for a given test protein which atoms are the most similar to atoms in a test set. 


In [18]:
import quippy
import ase
import numpy as np
import h5py
import matplotlib.pyplot as plt
import os
import collections 

%matplotlib notebook

In [ ]:
path_to_hdf5_file="samples2.xyz.hdf5"

In [3]:
dataFile = h5py.File("../samples2.xyz.hdf5", "r")
labels = dataFile["labels"]["label_mat"].value
average_simscore = generateAverageSimilarity(path_to_matrices, path_to_xyz_file, labels)

3kexA_nucleo


/home/wpg23/.local/miniconda3/envs/python2env/lib/python2.7/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


In [11]:
# Plot the average and std for each class, for the third protein
def generateAverageAndCounterSimilarity(path_to_matrices, path_to_xyz_file, labels):
    
    n_most_common = 10
    
    for index, _ in enumerate(labels):
        label = label[index][1]
        hemes = []
        nucleos = []
        heme_indices = []
        nucleo_indices = []
    
        for i, tag in enumerate(labels):
            kerns = np.load("{0}/{1}_{2}_kerns.npy".format(path_to_matrices,index,i))
            perms = np.load("{0}/{1}_{2}_perms.npy".format(path_to_matrices,index,i))
            atomic_similarity = kerns*perms
            # The similarity between each atom of the test protein and the target protein 
            atomic_similarity = np.sum(atomic_similarity, axis=1)
            indices = np.argsort(atomic_similarity)
            tag = tag[1]
            if "heme" in tag:
                hemes.append(atomic_similarity)
                heme_indices.append(indices)
            else:
                nucleos.append(atomic_similarity)
                nucleo_indices.append(indices)
        
        hemes = np.asarray(hemes)
        nucleos = np.asarray(nucleos)
        heme_means = np.mean(hemes, axis=0)
        nucleo_means = np.mean(nucleos, axis=0)

        # scale to 1, as otherwise the b-factors round out
        heme_means /= heme_means.max()
        nucleo_means /= nucleo_means.max()

        atom_reader = quippy.AtomsReader(path_to_xyz_file, format="xyz")
        # Crude, but quick enough for now
        for temp_protein in atom_reader:
            if temp_protein.params["tag"] == label:
                protein = temp_protein
                break
        else:
            print("Protein not found!")
            raise ValueError
        
        # Check that the arrays are the correct length, and referencing the correct protein
        assert protein.params["tag"] == label
        assert len(heme_means) == len(protein.arrays["positions"])
        assert len(nucleo_means) == len(protein.arrays["positions"])

        
        counter = collections.Counter()
        for row in heme_indices:
            counter.update(row[-n_most_common:])

        heme_counts = np.zeros(len(protein.arrays["positions"]))
        for index, count in counter.most_common():
            heme_counts[index] = count

        counter = collections.Counter()
        for row in nucleo_indices:
            counter.update(row[-n_most_common:])

        nucleo_counts = np.zeros(len(protein.arrays["positions"]))
        for index, count in counter.most_common():
            nucleo_counts[index] = count
            
            
        assert len(nucleo_counts) == len(nucleo_means)
        assert len(heme_counts) == len(heme_means)
        
        # We have the count-based and average-based similarity scores.
        # Now we need to obtain each atom's distance from the ligand.
        

In [ ]:
# arrays = []
# for i, tag in enumerate(labels):
#     kerns = np.load("basematrices/{0}_{1}_kerns.npy".format(index,i))
#     perms = np.load("basematrices/{0}_{1}_perms.npy".format(index,i))
#     atomic_similarity = kerns*perms
#     arrays.append(atomic_similarity)

# array_dict = dict(zip([x[1] for x in labels], arrays))
# np.savez_compressed(file="150", **array_dict)